In [1]:
1

1

In [9]:
!pip install --upgrade pip
!pip install paramiko

# !pip install gnupg


  Using cached paramiko-3.4.1-py3-none-any.whl (226 kB)
     |████████████████████████████████| 856 kB 5.5 MB/s            
     |████████████████████████████████| 3.7 MB 61.2 MB/s            
     |████████████████████████████████| 593 kB 118.4 MB/s            
  Attempting uninstall: cryptography
    Found existing installation: cryptography 2.7
    Uninstalling cryptography-2.7:
      Successfully uninstalled cryptography-2.7


In [10]:
import io
import os
import re
import glob
import pandas as pd
import numpy as np

from datetime import datetime
import datetime as dt
import time
import boto3

import zipfile
import pytz

from datetime import date, timedelta
import gzip
import pandas as pd
from pandas.io.json import json_normalize 
import json
import gnupg
from io import StringIO
import boto3
import paramiko
import os
import requests
import logging
import os
import typing
from os.path import join, dirname
import time
import logging
from paramiko import SFTPClient, SFTPFile, Message, SFTPError, Transport
from paramiko.sftp import CMD_STATUS, CMD_READ, CMD_DATA

logger = logging.getLogger('demo')


In [31]:
class _SFTPFileDownloader:
    """
    Helper class to download large file with paramiko sftp client with limited number of concurrent requests.
    """

    _DOWNLOAD_MAX_REQUESTS = 48
    _DOWNLOAD_MAX_CHUNK_SIZE = 0x8000

    def __init__(self, f_in: SFTPFile, f_out: typing.BinaryIO, callback=None):
        self.f_in = f_in
        self.f_out = f_out
        self.callback = callback

        self.requested_chunks = {}
        self.received_chunks = {}
        self.saved_exception = None

    def download(self):
        file_size = self.f_in.stat().st_size
        requested_size = 0
        received_size = 0

        while True:
            # send read requests
            while len(self.requested_chunks) + len(self.received_chunks) < self._DOWNLOAD_MAX_REQUESTS and \
                    requested_size < file_size:
                chunk_size = min(self._DOWNLOAD_MAX_CHUNK_SIZE, file_size - requested_size)
                request_id = self._sftp_async_read_request(
                    fileobj=self,
                    file_handle=self.f_in.handle,
                    offset=requested_size,
                    size=chunk_size
                )
                self.requested_chunks[request_id] = (requested_size, chunk_size)
                requested_size += chunk_size

            # receive blocks if they are available
            # note: the _async_response is invoked
            self.f_in.sftp._read_response()
            self._check_exception()

            # write received data to output stream
            while True:
                chunk = self.received_chunks.pop(received_size, None)
                if chunk is None:
                    break
                _, chunk_size, chunk_data = chunk
                self.f_out.write(chunk_data)
                if self.callback is not None:
                    self.callback(chunk_data)

                received_size += chunk_size

            # check transfer status
            if received_size >= file_size:
                break

            # check chunks queues
            if not self.requested_chunks and len(self.received_chunks) >= self._DOWNLOAD_MAX_REQUESTS:
                raise ValueError("SFTP communication error. The queue with requested file chunks is empty and"
                                 "the received chunks queue is full and cannot be consumed.")

        return received_size

    def _sftp_async_read_request(self, fileobj, file_handle, offset, size):
        sftp_client = self.f_in.sftp

        with sftp_client._lock:
            num = sftp_client.request_number

            msg = Message()
            msg.add_int(num)
            msg.add_string(file_handle)
            msg.add_int64(offset)
            msg.add_int(size)

            sftp_client._expecting[num] = fileobj
            sftp_client.request_number += 1

        sftp_client._send_packet(CMD_READ, msg)
        return num

    def _async_response(self, t, msg, num):
        if t == CMD_STATUS:
            # save exception and re-raise it on next file operation
            try:
                self.f_in.sftp._convert_status(msg)
            except Exception as e:
                self.saved_exception = e
            return
        if t != CMD_DATA:
            raise SFTPError("Expected data")
        data = msg.get_string()

        chunk_data = self.requested_chunks.pop(num, None)
        if chunk_data is None:
            return

        # save chunk
        offset, size = chunk_data

        if size != len(data):
            raise SFTPError(f"Invalid data block size. Expected {size} bytes, but it has {len(data)} size")
        self.received_chunks[offset] = (offset, size, data)

    def _check_exception(self):
        """if there's a saved exception, raise & clear it"""
        if self.saved_exception is not None:
            x = self.saved_exception
            self.saved_exception = None
            raise x


def download_file(sftp_client: SFTPClient, remote_path: str, local_path: str, callback=None):
    """
    Helper function to download remote file via sftp.
    It contains a fix for a bug that prevents a large file downloading with :meth:`paramiko.SFTPClient.get`
    Note: this function relies on some private paramiko API and has been tested with paramiko 2.7.1.
          So it may not work with other paramiko versions.
    :param sftp_client: paramiko sftp client
    :param remote_path: remote file path
    :param local_path: local file path
    :param callback: optional data callback
    """
    remote_file_size = sftp_client.stat(remote_path).st_size

    with sftp_client.open(remote_path, 'rb') as f_in, open(local_path, 'wb') as f_out:
        _SFTPFileDownloader(
            f_in=f_in,
            f_out=f_out,
            callback=callback
        ).download()

    local_file_size = os.path.getsize(local_path)
    if remote_file_size != local_file_size:
        raise IOError(f"file size mismatch: {remote_file_size} != {local_file_size}")


def main():

    start_time = time.time()
    # public sftp server
    host = 'DOTS.datalabusa.com'
    port = 22
    username = 'ex_Avant'
    password = 'Et#h@ut8Ec2F'
    
    ### Ignore file list    
    s3_resource = boto3.resource("s3") 
    my_bucket = s3_resource.Bucket('avant-partner01-landing')
    list1 = []
    for my_bucket_object in my_bucket.objects.filter(Prefix='datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410'):
        a = my_bucket_object.key.replace('datalabs/drop/interim/','')
        list1.append(a)

    for my_bucket_object in my_bucket.objects.filter(Prefix='datalabs/drop/interim/DL_AVT_INTERIMFILE2_202410'):
        a = my_bucket_object.key.replace('datalabs/drop/interim/','')
        list1.append(a)

    ### Getting file list
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    ssh.connect(hostname=host,port=port,username=username,password=password)
    sftp = ssh.open_sftp()

    file_list = sftp.listdir('From_Datalab')
    campaign_month = '202410' 
    campaign_files = [file for file in file_list if campaign_month in file and file not in list1]
    
    ssh.close()
    sftp.close()
    
    
    for file in campaign_files:
    
        remote_file_path = 'From_Datalab/{}'.format(file)
    #     local_file_path = join(dirname(__file__), 'DL_AVT_INTERIMFILE2_202408_HI.csv.pgp')
        local_file_path = '/tmp/{}'.format(file)

        transport = Transport((host, port))
        transport.set_keepalive(30)
        transport.connect(
            username=username,
            password=password,
        )

        with SFTPClient.from_transport(transport) as sftp_client:
            progress_size = 0
            total_size = 0
            step_size = 4 * 1024 * 1024

            def progress_callback(data):
                nonlocal progress_size, total_size
                progress_size += len(data)
                total_size += len(data)
                while progress_size >= step_size:
                    logger.info(f"{total_size // (1024 ** 2)} MB has been downloaded")
                    progress_size -= step_size

            download_file(
                sftp_client=sftp_client,
                remote_path=remote_file_path,
                local_path=local_file_path,
                callback=progress_callback
            )
        download_time = time.time() - start_time
        print(f"File downloaded successfully to {local_file_path} in {download_time:.2f} seconds")

        # s4 upload
        s3_bucket_name = 'avant-partner01-landing'
        s3_object_key = 'datalabs/drop/interim/{file}'.format(
        campaign = campaign_month,
        file = file
        )
        s3_resource = boto3.resource("s3")
        s3_resource.Bucket(name=s3_bucket_name).upload_file(
        Filename=local_file_path, Key=s3_object_key
        )


        # Clean up local file
        os.remove(local_file_path)
    
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")
    main()



2024-08-25 15:31:49,342 [INFO]: Connected (version 2.0, client MOVEit)
2024-08-25 15:31:49,993 [INFO]: Authentication (password) successful!
2024-08-25 15:31:50,060 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-08-25 15:31:50,277 [INFO]: [chan 0] sftp session closed.
2024-08-25 15:31:50,324 [INFO]: Connected (version 2.0, client MOVEit)
2024-08-25 15:31:51,015 [INFO]: Authentication (password) successful!
2024-08-25 15:31:51,084 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-08-25 15:31:51,939 [INFO]: 4 MB has been downloaded
2024-08-25 15:31:52,544 [INFO]: 8 MB has been downloaded
2024-08-25 15:31:53,171 [INFO]: 12 MB has been downloaded
2024-08-25 15:31:53,736 [INFO]: 16 MB has been downloaded
2024-08-25 15:31:54,391 [INFO]: 20 MB has been downloaded
2024-08-25 15:31:55,006 [INFO]: 24 MB has been downloaded
2024-08-25 15:31:55,632 [INFO]: 28 MB has been downloaded
2024-08-25 15:31:56,280 [INFO]: 32 MB has been downloaded
2024-08-25 15:31:56,905 [INF

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE1_202410_5.csv.pgp in 228.86 seconds


2024-08-25 15:35:44,488 [INFO]: Connected (version 2.0, client MOVEit)
2024-08-25 15:35:45,338 [INFO]: Authentication (password) successful!
2024-08-25 15:35:45,410 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-08-25 15:35:46,340 [INFO]: 4 MB has been downloaded
2024-08-25 15:35:46,955 [INFO]: 8 MB has been downloaded
2024-08-25 15:35:47,528 [INFO]: 12 MB has been downloaded
2024-08-25 15:35:48,047 [INFO]: 16 MB has been downloaded
2024-08-25 15:35:48,613 [INFO]: 20 MB has been downloaded
2024-08-25 15:35:49,154 [INFO]: 24 MB has been downloaded
2024-08-25 15:35:49,755 [INFO]: 28 MB has been downloaded
2024-08-25 15:35:50,323 [INFO]: 32 MB has been downloaded
2024-08-25 15:35:50,851 [INFO]: 36 MB has been downloaded
2024-08-25 15:35:51,415 [INFO]: 40 MB has been downloaded
2024-08-25 15:35:51,976 [INFO]: 44 MB has been downloaded
2024-08-25 15:35:52,533 [INFO]: 48 MB has been downloaded
2024-08-25 15:35:53,096 [INFO]: 52 MB has been downloaded
2024-08-25 15:35:53,659 

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE1_202410_6.csv.pgp in 465.07 seconds


2024-08-25 15:39:40,868 [INFO]: Connected (version 2.0, client MOVEit)
2024-08-25 15:39:41,435 [INFO]: Authentication (password) successful!
2024-08-25 15:39:41,501 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-08-25 15:39:42,306 [INFO]: 4 MB has been downloaded
2024-08-25 15:39:42,869 [INFO]: 8 MB has been downloaded
2024-08-25 15:39:43,415 [INFO]: 12 MB has been downloaded
2024-08-25 15:39:43,988 [INFO]: 16 MB has been downloaded
2024-08-25 15:39:44,540 [INFO]: 20 MB has been downloaded
2024-08-25 15:39:45,097 [INFO]: 24 MB has been downloaded
2024-08-25 15:39:45,660 [INFO]: 28 MB has been downloaded
2024-08-25 15:39:46,233 [INFO]: 32 MB has been downloaded
2024-08-25 15:39:46,785 [INFO]: 36 MB has been downloaded
2024-08-25 15:39:47,366 [INFO]: 40 MB has been downloaded
2024-08-25 15:39:47,910 [INFO]: 44 MB has been downloaded
2024-08-25 15:39:48,472 [INFO]: 48 MB has been downloaded
2024-08-25 15:39:49,035 [INFO]: 52 MB has been downloaded
2024-08-25 15:39:49,605 

...

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE2_202410_7.csv.pgp in 7565.53 seconds


2024-08-25 17:38:04,966 [INFO]: Connected (version 2.0, client MOVEit)
2024-08-25 17:38:05,630 [INFO]: Authentication (password) successful!
2024-08-25 17:38:05,698 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-08-25 17:38:06,547 [INFO]: 4 MB has been downloaded
2024-08-25 17:38:07,143 [INFO]: 8 MB has been downloaded
2024-08-25 17:38:07,759 [INFO]: 12 MB has been downloaded
2024-08-25 17:38:08,359 [INFO]: 16 MB has been downloaded
2024-08-25 17:38:08,918 [INFO]: 20 MB has been downloaded
2024-08-25 17:38:09,519 [INFO]: 24 MB has been downloaded
2024-08-25 17:38:10,082 [INFO]: 28 MB has been downloaded
2024-08-25 17:38:10,746 [INFO]: 32 MB has been downloaded
2024-08-25 17:38:11,319 [INFO]: 36 MB has been downloaded
2024-08-25 17:38:11,938 [INFO]: 40 MB has been downloaded
2024-08-25 17:38:12,459 [INFO]: 44 MB has been downloaded
2024-08-25 17:38:13,004 [INFO]: 48 MB has been downloaded
2024-08-25 17:38:13,600 [INFO]: 52 MB has been downloaded
2024-08-25 17:38:14,156 

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE2_202410_8.csv.pgp in 7950.93 seconds


2024-08-25 17:44:30,346 [INFO]: Connected (version 2.0, client MOVEit)
2024-08-25 17:44:31,267 [INFO]: Authentication (password) successful!
2024-08-25 17:44:31,341 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-08-25 17:44:32,203 [INFO]: 4 MB has been downloaded
2024-08-25 17:44:32,804 [INFO]: 8 MB has been downloaded
2024-08-25 17:44:33,456 [INFO]: 12 MB has been downloaded
2024-08-25 17:44:34,039 [INFO]: 16 MB has been downloaded
2024-08-25 17:44:34,655 [INFO]: 20 MB has been downloaded
2024-08-25 17:44:35,271 [INFO]: 24 MB has been downloaded
2024-08-25 17:44:35,882 [INFO]: 28 MB has been downloaded
2024-08-25 17:44:36,503 [INFO]: 32 MB has been downloaded
2024-08-25 17:44:37,114 [INFO]: 36 MB has been downloaded
2024-08-25 17:44:37,695 [INFO]: 40 MB has been downloaded
2024-08-25 17:44:38,291 [INFO]: 44 MB has been downloaded
2024-08-25 17:44:38,936 [INFO]: 48 MB has been downloaded
2024-08-25 17:44:39,513 [INFO]: 52 MB has been downloaded
2024-08-25 17:44:40,145 

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE2_202410_9.csv.pgp in 8341.60 seconds


In [30]:
list1

['DL_AVT_INTERIMFILE1_202410_1.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_10.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_11.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_12.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_13.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_14.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_15.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_16.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_17.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_18.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_19.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_2.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_3.csv.pgp',
 'DL_AVT_INTERIMFILE1_202410_4.csv.pgp']

In [32]:
### Ignore file list    
s3_resource = boto3.resource("s3") 
my_bucket = s3_resource.Bucket('avant-partner01-landing')
list1 = []
for my_bucket_object in my_bucket.objects.filter(Prefix='datalabs/drop/interim/DL_AVT_INTERIMFILE1_202410'):
    a = my_bucket_object.key.replace('datalabs/drop/interim/','')
    list1.append(a)

for my_bucket_object in my_bucket.objects.filter(Prefix='datalabs/drop/interim/DL_AVT_INTERIMFILE2_202410'):
    a = my_bucket_object.key.replace('datalabs/drop/interim/','')
    list1.append(a)

In [34]:
len(list1)

38